### Week 4 - Labs Notebook 2 - Multiple Sequence Alignment
- October 2023
- [https://https://github.com/tisimpson/bioinformatics1](https://github.com/tisimpson/bioinformatics1)
- [ian.simpson@ed.ac.uk](mailto:ian.simpson@ed.ac.uk)

In this notebook we are going to walk through an experiment where we retreive sequences, do BLAST searches and then use the results to format and then execute a multiple sequence alignment using the MUSCLE software package. At the end we even create a basic phylogenetic tree from the alignment and then visualise it.

The setup of the MUSCLE aligner is a little technical so we strongly reccommend that if you want to use it you install it using conda using the command ``conda install -c bioconda muscle``

### Part 1 : Settting Up

In [ ]:
%pip install biopython

# import required Biophython functions 
from Bio import Entrez
from Bio.Blast import NCBIXML
from Bio.Blast import NCBIWWW
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align.Applications import MuscleCommandline
from Bio import AlignIO
from Bio.Align import AlignInfo
from matplotlib import pyplot as plt

### Part 2 : Obtaining Sequences from NCBI

We're going to do this with Protein sequences, best to go to NCBI on the web to make sure you call for the correct sequence. The sequence accession should strt with "NP_".

In [ ]:
Entrez.email = 'ian.simpson@ed.ac.uk'

# beta-globin, human
my_protein = 'NP_000509.1' 

handle = Entrez.efetch(db="protein", id=my_protein, rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank")
handle.close()

# show the sequence record
print(record)

### Part 3 : Find Related Sequences Using BLAST

In [ ]:
result_handle = NCBIWWW.qblast('blastp', 'swissprot', record.seq)
# This may take some time to run

# parse the results
result_handle.seek(0)
blast_record = NCBIXML.read(result_handle)

To see the data structure for the results, go [here](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc94)

In [ ]:
#print out the key information for the hits

print('Gene name\te-value\tscore')
for a in blast_record.alignments:
    print(a.title.split('|')[2].split('Full=')[1].split(';')[0]+'\t'+str(a.hsps[0].expect)+'\t'+str( a.hsps[0].score))


In [ ]:
# show the species and alignment scores
a=blast_record.alignments[0]
sp_ids = []
for a in blast_record.alignments:
    sp_ids.append(a.title.split('|')[1])
# print(",".join(sp_ids))
handle = Entrez.efetch(db="protein", id=",".join(sp_ids), retmode="xml")#, rettype='gb')
data = Entrez.read(handle)
species = []
print('Alignment score\tSpecies')
for i,d in enumerate(data):
    species.append(d['GBSeq_source'])
    print(str(blast_record.alignments[i].hsps[0].score)+'\t'+d['GBSeq_source'])

### Part Four : Select Sequences for Multiple Sequence Alignment

Select sequences based on an e-value threshold then for each selected sequence print out:-
- name of alignment
- length of alignment
- e-value
- Query sequence
- Matching sequence
- Alignment info


In [ ]:
E_VALUE_THRESH = 1e-6

for i,alignment in enumerate(blast_record.alignments):
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print('****Alignment****')
            print('sequence: ', alignment.title)
            print('species: '+species[i])
            print('length: ', alignment.length)
            print('e value: ', hsp.expect)
            print(hsp.query[0:75] + '...')
            print(hsp.sbjct[0:75] + '...')
            print(hsp.match[0:75] + '...')


### Part Five : Create FASTA File of Sequences for Alignment

In [ ]:
# now work with all results with e-value below this value:
E_VALUE_THRESH = 1e-6

# the following will write all results into a FASTA file for the MSA 

def get_seqrecs(alignments, threshold):
    # a little helper function to get the sequence records
    for i,aln in enumerate(alignments):
        for hsp in aln.hsps:
            if hsp.expect < threshold:
                id = species[i]
                # id = aln.title.split('|')[1].split(' ')[0].split('_')[0]+'_'+species[i].replace(' ','_')
                print(id)
                yield SeqRecord(Seq(hsp.sbjct), id=id,description=str(aln.title.split('|')[1]))
                break
 
best_seqs = get_seqrecs(blast_record.alignments, E_VALUE_THRESH)
# write out to a fasta file
SeqIO.write(best_seqs, 'blast_selected_globins.fa', 'fasta')

### Part Six : Run MUSCLE Alignment

In [ ]:
import os

# run Muscle MSA
cmdLine = 'muscle -align blast_selected_globins.fa -output blast_selected_globins_alignment.aln'
os.popen(cmdLine)

In [ ]:
#read in and then print out alignment
alignment = AlignIO.read('blast_selected_globins_alignment.aln','fasta')
print(alignment)

In [ ]:
summary_align = AlignInfo.SummaryInfo(alignment)

# compute a consensus sequence by taking the most frequent letter
# positions below a thresold similarity are shown as 'X'

# the threshold can be adjusted by adding e.g. threshold=0.5

print('Consensus sequence without gaps:')
print(summary_align.dumb_consensus())
print('Consensus sequence with gaps:')
print(summary_align.gap_consensus())

In [ ]:
# print a Position Specific Score Matrix (PSSM)
# this shows the number of letters counted at each locationa./mu    
# in the sequence, which is shown in vertical along the left
pssm = summary_align.pos_specific_score_matrix(summary_align.dumb_consensus(), chars_to_ignore = ['X'])
print(pssm)

### Part Seven : Distantly Related Globins

In [ ]:
# run Muscle MSA
import os

# run Muscle MSA
cmdLine = 'muscle -align globins.fa -output distant_globins.aln'
os.popen(cmdLine)

#read in and then print out alignment
alignment = AlignIO.read('distant_globins.aln','fasta')

#print out the alignment, note I am printing it using the CLUSTAL X file format, this is still a MUSCLE alignment
print(format(alignment,'clustal'))

### Part 8 : Build a UPGMA tree

In [ ]:
from Bio.Phylo.TreeConstruction import DistanceCalculator
#from TreeConstruction import DistanceCalculator
calculator = DistanceCalculator('identity')
dm = calculator.get_distance(alignment)
print(dm)

In [ ]:
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
#from TreeConstruction import DistanceTreeConstructor
# here supply the keyword upgma or nj
# compare the trees you get from both methods
constructor = DistanceTreeConstructor(calculator, 'upgma')
tree = constructor.build_tree(alignment)
print(tree)

In [ ]:
from Bio import Phylo
# now draw the tree, try out these three methods:
Phylo.draw_ascii(tree)

In [ ]:
# or a nicer looking one
plt.figure(figsize=(12,12))
ax=plt.subplot(111)
Phylo.draw(tree,axes=ax)